**Objetivo**: descobrir volumes de árvores usando regressao linear de um modelo de rede neural

In [2]:
#instalação de pacotes necessários
#install.packages("e1071")
#install.packages("caret")
#install.packages("mlbench")
#install.packages("mice")
#install.packages("Metrics")
library(caret)
library(mlbench)
library(mice)
library(Metrics)

Carregando pacotes exigidos: ggplot2

Carregando pacotes exigidos: lattice


Anexando pacote: 'mice'


O seguinte objeto é mascarado por 'package:stats':

    filter


Os seguintes objetos são mascarados por 'package:base':

    cbind, rbind



Anexando pacote: 'Metrics'


Os seguintes objetos são mascarados por 'package:caret':

    precision, recall




In [3]:
#leitura de arquivo de dados

dados <- read.csv("databases/2 - Volume - Dados.csv")

head(dados)

,PROJETO,CLONE,Idade,Hdom,AreaBasal,Volume
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,Fazenda Velha,C3,21.5,8.8,3.2,12.1
2,Fazenda Velha,A3,21.9,10.2,3.9,66.8
3,Fazenda Velha,A3,26.1,10.1,4.2,72.1
4,Fazenda Velha,A4,22.0,10.0,4.2,35.9
5,Parada,B7,23.4,15.3,4.3,44.5
6,Fazenda Velha,A2,25.9,10.5,4.4,63.9


In [4]:
## separa em treino e teste

set.seed(123)

indices <- createDataPartition(dados$Volume, p = 0.80, list = FALSE)

treino <- dados[indices, ]
teste <- dados[-indices, ]

In [5]:
#treino vcom hold-out
set.seed(123)
rna <- train(Volume ~ ., data = treino, method = "nnet",
             linout = TRUE, trace = FALSE)
rna
predict_rna <- predict(rna, teste)

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
"There were missing values in resampled performance measures."


Neural Network 

246 samples
  5 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 246, 246, 246, 246, 246, 246, ... 
Resampling results across tuning parameters:

  size  decay  RMSE       Rsquared   MAE      
  1     0e+00  196.54992  0.2860598  166.11881
  1     1e-04  192.44537  0.3423822  163.06421
  1     1e-01  124.48168  0.6155058  101.92691
  3     0e+00  178.20292  0.4346005  150.45010
  3     1e-04  168.55044  0.5836749  141.81632
  3     1e-01   82.87502  0.7824127   65.25148
  5     0e+00  149.33581  0.4601841  124.63413
  5     1e-04  157.63349  0.5506802  129.37686
  5     1e-01   78.23198  0.8097598   60.84637

RMSE was used to select the optimal model using the smallest value.
The final values used for the model were size = 5 and decay = 0.1.

In [6]:
rmse(teste$Volume, predict_rna)

r2 <- function(predito, observado) {
  return(1 - (sum((predito - observado)^2) /
              sum((observado - mean(observado))^2)))
}

r2(predict_rna, teste$Volume)

[1] 116.6268

[1] 0.6483077

In [7]:
# Cross-validation e parametrização da rna
ctrl <- trainControl(method = "cv", number = 10)
grid <- expand.grid(size = seq(from = 1, to = 10, by = 1),
                    decay = seq(from = 0.1, to = 0.9, by = 0.3))

set.seed(123)
rna <- train(Volume ~ ., data = treino, method = "nnet",
             trainControl = ctrl, tuneGrid = grid, linout = TRUE,
             MaxNWts = 10000, maxit = 2000, trace = FALSE)
rna

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
"There were missing values in resampled performance measures."


Neural Network 

246 samples
  5 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 246, 246, 246, 246, 246, 246, ... 
Resampling results across tuning parameters:

  size  decay  RMSE      Rsquared   MAE      
   1    0.1    44.24422  0.9871448  35.187445
   1    0.4    42.78998  0.9791873  32.958013
   1    0.7    39.19532  0.9748414  29.447295
   2    0.1    24.14329  0.9929071  18.379050
   2    0.4    20.23803  0.9890586  13.182328
   2    0.7    21.87821  0.9879347  14.212487
   3    0.1    14.10707  0.9944970   9.496477
   3    0.4    16.04145  0.9930650  10.518049
   3    0.7    19.04136  0.9906446  11.645061
   4    0.1    12.56667  0.9957220   8.666376
   4    0.4    14.81401  0.9941712   9.247001
   4    0.7    16.95705  0.9918499  10.466543
   5    0.1    13.11311  0.9953789   8.264301
   5    0.4    14.77824  0.9940014   9.215850
   5    0.7    16.41375  0.9930595   9.873928
   6    0.1    13.10989  0.9955879   8.208540
   6    0.4   

In [8]:
# predicoes e metricas
predict_rna <- predict(rna, teste)

rmse(teste$Volume, predict_rna)

r2(predict_rna, teste$Volume)


[1] 9.018405

[1] 0.9978971

In [9]:
##predição de novos casos

dados_novos_casos <- read.csv("databases/2 - Volume - Dados - Novos Casos.csv")

View(dados_novos_casos)

predict_rna <- predict(rna, dados_novos_casos)
dados_novos_casos$Volume <- NULL

resultado <- cbind(dados_novos_casos, predict_rna)

View(resultado)

PROJETO,CLONE,Idade,Hdom,AreaBasal,Volume
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
Parada,C3,21.5,8.8,4.2,?
Santa Maria,A3,22.9,14.1,5.6,?
Jacutinga,A2,59.4,22.3,13.0,?


,PROJETO,CLONE,Idade,Hdom,AreaBasal,predict_rna
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,Parada,C3,21.5,8.8,4.2,32.54981
2,Santa Maria,A3,22.9,14.1,5.6,132.13582
3,Jacutinga,A2,59.4,22.3,13.0,283.05599
